In [1]:
cd E:/GitHub/computervision

E:\GitHub\computervision


In [2]:
import torch
import os
# os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

from torch.nn import CrossEntropyLoss
from datetime import datetime
from torch.optim import lr_scheduler, SGD, Adamax
from modules.training_util import PytorchDataset, PytorchTraining, EarlyStopping
from modules.models_util import ViT

In [3]:
# Variables for continuing previous training
CONTINUE_TRAINING_DIR = None
# CONTINUE_TRAINING_DIR = 'notebooks/vit/models/'

MODEL_WEIGHTS_PATH = None

if CONTINUE_TRAINING_DIR:
    MODEL_WEIGHTS_PATH = os.path.join(CONTINUE_TRAINING_DIR,'epoch_15.pth')

# CONFIG VARIABLES
FULLY_TRAIN = True
base_path = 'E:/Datasets/cor-splits'
DATA_DIR = [
    # os.path.join(base_path,'sgkf-8-1-1-size-540'),
    # os.path.join(base_path,'balanced_train'),
    # os.path.join(base_path,'imbalanced_train'),
    # os.path.join(base_path,'full_imbalanced_train'),
    # os.path.join(base_path,'natural_imbalanced_train'),
    os.path.join(base_path,'underperforming_cameras_96')
    ]
START_EPOCH = 1 # Change it from 1 if you're continuing an existing run
N_EPOCHS = 50
BATCH_SIZE = 16
SAVE_INTERVAL = 4 # How many epochs between each backup
## Learning Rate Scheduler
LR = 0.001  # When continuing a existing run, update it to the last epoch LR value
LR_EPOCH_SCHEDULE = 10  # Number of epochs before altering LR
LR_GAMMA = 0.1
## Early Stopping
PATIENCE = LR_EPOCH_SCHEDULE + 1
MIN_DELTA = 0
MIN_EPOCH = 10 # Number of spochs before starting patience counter

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
archs = ['b32']

In [5]:
for arch in archs:
    for dir in DATA_DIR:
        vit = ViT(architecture=arch)
        
        model = vit.load(MODEL_WEIGHTS_PATH, FULLY_TRAIN)
        model = model.to(device)
        
        criterion = CrossEntropyLoss()

        if FULLY_TRAIN:
            params = model.parameters()
        else:
            params = model.heads.parameters()
            
        # optimizer = optim.SGD(model.classifier.parameters(), lr=LR)
        optimizer = SGD(params, lr=LR)
        # optimizer = Adamax(model.parameters(), lr=LR)

        step_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=LR_EPOCH_SCHEDULE, gamma=LR_GAMMA)
        es = EarlyStopping(patience=PATIENCE, delta=MIN_DELTA, min_epoch=MIN_EPOCH)
        dataset = PytorchDataset(dir, vit.data_transforms, BATCH_SIZE)

        working_folder = 'notebooks/vit'
        models_folder = 'models'

        curr_time = datetime.now()
        curr_time = curr_time.strftime("%Y-%m-%d %H-%M-%S")

        output_folder = os.path.join(working_folder, models_folder)
        output_folder = os.path.join(output_folder, curr_time)
        
        if not os.path.exists(output_folder):
            os.makedirs(output_folder)

        trainer = PytorchTraining(device, dataset, output_folder)
        model = trainer.train_pytorch_model(model, criterion, optimizer, step_lr_scheduler, es, START_EPOCH, N_EPOCHS, SAVE_INTERVAL, arch)

All parameters for model VisionTransformer requires grad.
Saving model to folder notebooks/vit\models\2024-03-07 16-43-10

Model VisionTransformer b32
Fully Trained = True
Dataset E:/Datasets/cor-splits\underperforming_cameras_96
Learning Rate Epoch Schedule = 10
Learning Rate Gamma = 0.1
Optimizer = SGD
Batch Size = 16


Epoch 1/50
----------



c:\Users\Adm\anaconda3\envs\computervision\Lib\site-packages\torch\nn\functional.py:5476: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = scaled_dot_product_attention(q, k, v, attn_mask, dropout_p, is_causal)


train Loss: 0.6062 Acc: 0.6680
val Loss: 0.4828 Acc: 0.8103
Epoch duration: 1 m 17s
Learning Rate = 0.001



Epoch 2/50
----------

train Loss: 0.4021 Acc: 0.8387
val Loss: 0.3966 Acc: 0.8308
Epoch duration: 1 m 14s
Learning Rate = 0.001



Epoch 3/50
----------

train Loss: 0.2704 Acc: 0.8991
val Loss: 0.4427 Acc: 0.8115
Epoch duration: 1 m 14s
Learning Rate = 0.001



Epoch 4/50
----------

train Loss: 0.1817 Acc: 0.9370
val Loss: 0.3854 Acc: 0.8269
Epoch duration: 1 m 15s
Learning Rate = 0.001



Epoch 5/50
----------

train Loss: 0.1239 Acc: 0.9589
val Loss: 0.4135 Acc: 0.8256
Epoch duration: 1 m 14s
Learning Rate = 0.001



Epoch 6/50
----------

train Loss: 0.0830 Acc: 0.9713
val Loss: 0.4084 Acc: 0.8423
Epoch duration: 1 m 11s
Learning Rate = 0.001



Epoch 7/50
----------

train Loss: 0.0644 Acc: 0.9791
val Loss: 0.4386 Acc: 0.8231
Epoch duration: 1 m 13s
Learning Rate = 0.001



Epoch 8/50
----------

train Loss: 0.0540 Acc: 0.9840
val Loss: 0.4178 Acc: 0.8423
Epoch duration: 